In [3]:
%pip install onnx onnxruntime

     ---------------------------------------- 13.3/13.3 MB 3.4 MB/s eta 0:00:00
     ---------------------------------------- 6.7/6.7 MB 3.7 MB/s eta 0:00:00
     ---------------------------------------- 46.0/46.0 kB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 86.8/86.8 kB 4.8 MB/s eta 0:00:00
     ---------------------------------------- 95.2/95.2 kB 5.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import torch
from torch import nn
import onnxruntime as onnxrt
import sdae, sdae_1, sdae_2

LR = 1e-3
WEIGHT_DECAY = 1e-5
LOSS_FUNC = nn.MSELoss()
IN_CHANNELS = 3

proper_model = sdae.StackedDenoisingAutoencoder(IN_CHANNELS, LOSS_FUNC, LR, WEIGHT_DECAY)
overfit_model = sdae_1.StackedDenoisingAutoencoder(IN_CHANNELS, LOSS_FUNC, LR, WEIGHT_DECAY)
underfit_model = sdae_2.StackedDenoisingAutoencoder(IN_CHANNELS, LOSS_FUNC, LR, WEIGHT_DECAY)

In [7]:
underfit_model

StackedDenoisingAutoencoder(
  (dae_1): DenoisingAutoencoder(
    (encoder): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (4): ReLU()
    )
    (decoder): Sequential(
      (0): ConvTranspose2d(32, 3, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), output_padding=(1, 1))
      (1): ReLU()
    )
    (criterion): MSELoss()
  )
  (dae_2): DenoisingAutoencoder(
    (encoder): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (4): ReLU()
    )
    (decoder): Sequential(
      (0): ConvTranspose2d(64, 32, ke

In [2]:
dummy_input = torch.randn(32, 3, 256, 256)
input_names = ["actual_input"]
output_names = ["encoder_output", "decoder_output"]

In [3]:
torch.onnx.export(
    proper_model,
    dummy_input,
    "sdae_model_final.onnx",
    input_names=input_names,
    output_names=output_names,
    export_params=True,
)

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [4]:
torch.onnx.export(
    overfit_model,
    dummy_input,
    "sdae_model_overfit.onnx",
    input_names=input_names,
    output_names=output_names,
    export_params=True,
)

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [5]:
torch.onnx.export(
    underfit_model,
    dummy_input,
    "sdae_model_underfit.onnx",
    input_names=input_names,
    output_names=output_names,
    export_params=True,
)

================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================

